In [1]:
!pip install beautifulsoup4

In [12]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
from itertools import chain
from collections import Counter

In [13]:
url_list = ['https://edition.cnn.com/2022/11/17/tech/twitter-employees-ultimatum-deadline/index.html',
'https://edition.cnn.com/2022/11/17/india/modi-india-g20-influence-intl-hnk/index.html',
'https://www.dailymail.co.uk/sport/sportsnews/article-11442203/Qatar-want-alcohol-sales-World-Cup-stadiums-BANNED.html',
'http://edition.cnn.com/2022/11/18/us/five-things-november-18-trnd/index.html',
'https://www.dailymail.co.uk/sport/sportsnews/article-11443573/Man-United-respond-Cristiano-Ronaldo-interview-Piers-Morgan.html',
'https://www.dailymail.co.uk/femail/article-11443077/This-buttons-left-womens-shirts.html',
'https://www.dailymail.co.uk/tvshowbiz/article-11443607/Lorraine-Kelly-shocks-fellow-celebs-expletive-ridden-rant-hilarious-Children-Need-sketch.html']

In [14]:
# url = 'https://edition.cnn.com/2022/11/17/tech/twitter-employees-ultimatum-deadline/index.html'
# url = 'https://edition.cnn.com/2022/11/17/india/modi-india-g20-influence-intl-hnk/index.html'
# url = 'https://www.dailymail.co.uk/sport/sportsnews/article-11442203/Qatar-want-alcohol-sales-World-Cup-stadiums-BANNED.html'
url = 'http://edition.cnn.com/2022/11/18/us/five-things-november-18-trnd/index.html'
# url = 'https://www.dailymail.co.uk/sport/sportsnews/article-11443573/Man-United-respond-Cristiano-Ronaldo-interview-Piers-Morgan.html'
# url = 'https://www.dailymail.co.uk/femail/article-11443077/This-buttons-left-womens-shirts.html'
# url = 'https://www.dailymail.co.uk/tvshowbiz/article-11443607/Lorraine-Kelly-shocks-fellow-celebs-expletive-ridden-rant-hilarious-Children-Need-sketch.html'

response = requests.get(url)

In [15]:
soup = BeautifulSoup(response.content, 'html.parser')

In [16]:
# text = soup.find_all('p', class_='paragraph inline-placeholder')
text = soup.find_all('p')
text_ = [item.text.strip() for item in text]
text_ = ''.join(text_)

In [17]:
def CNN_scraper(url):
    """
    Input: 'str'
    Output: 'dict'
    
    The function recieve an url (must be from CNN), fetch for the html and uses BS4 to extract the paragraph
    with the class='paragraph inline-placeholder' which contains the text. Then clean and merge the strings.
    It returns a dictionary with the title and the text of the news
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.find_all('p', class_='paragraph inline-placeholder')
    text = [item.text.strip() for item in text]
    text = ''.join(text).replace('\xa0', ' ')
    title = soup.title.string.split('|')[0]

    return {'title': title, 'article': text, 'id': 0, 'orig_id': 0}

In [18]:
def DailyMail_scraper(url):
    """
    Input: 'str'
    Output: 'dict'
    
    The function recieve an url (must be from DailyMail), fetch for the html and uses BS4 to extract the paragraph
    with the class='paragraph inline-placeholder' which contains the text. Then clean and merge the strings.
    It returns a dictionary with the title and the text of the news
    """
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.find_all('p', class_='mol-para-with-font')
    text = [item.text.strip() for item in text]
    text = ''.join(text).replace('\xa0', ' ')
    title = soup.title.string.split('|')[0]

    return {'title': title, 'article': text, 'id': 0, 'orig_id': 0}

In [19]:
def General_scraper(url):
    """
    Input: 'str'
    Output: 'dict'
    
    The function recieve an url, fetch for the html and uses BS4 to extract the paragraph tags. Then it
    counts the number of times that each paragraph is repeated and uses the most repeated (in a news must be text)
    to scrape the news from the website. It returns a dictionary with the title and the text of the news.
    """
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    text = soup.find_all('p')

    attrs_list = [' '.join(item.attrs['class']) for item in text if len(item.attrs) > 0]
    values, counts = np.unique(attrs_list, return_counts=True)
    idx = np.where(counts == np.max(counts))
    text_class = str(values[idx]).strip('[]\'')

    text = soup.find_all('p', class_=text_class)
    text = [item.text.strip() for item in text]
    text = ' '.join(text).replace('\xa0', ' ')
    title = soup.title.string.split('|')[0]

    return {'title': title, 'article': text, 'id': 0, 'orig_id': 0}

In [20]:
v1 = General_scraper(url)
v2 = DailyMail_scraper(url)
v3 = CNN_scraper(url)

In [21]:
print('Comparison General to DailyMail:',v1['article'] == v2['article'])
print('Comparison General to CNN:', v1['article'] == v3['article'])

Comparison General to DailyMail: False
Comparison General to CNN: False


In [22]:
count = 0

for link in url_list:
    v1 = General_scraper(url)
    v2 = DailyMail_scraper(url)
    v3 = CNN_scraper(url)

    c1 = v1['article'] == v2['article']
    c2 = v1['article'] == v3['article']

    count = count + c1 + c2

print(f'Total count of comparable text between scrapers: {count} and total number of url: {len(url_list)}')


Total count of comparable text between scrapers: 0 and total number of url: 7
